In [1]:
import os
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
import time
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn.metrics import classification_report
import joblib
import re

In [2]:
np.random.seed(50)
def load_data(dataset_name):
    if dataset_name == 'Iris':
        dataset = datasets.load_iris()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = dataset.target
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Banknote':
        df = pd.read_csv('./datasets/banknote_authentication.csv') 
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        #targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        targets = df['target'].values
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Blood_Transfusion':
        df = pd.read_csv('./datasets/blood_transfusion.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Breast_Cancer':
        dataset = datasets.load_breast_cancer()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data) 
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Climate':
        df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = df['target'].values
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Column':
        df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
        df['target']=np.where(df['target']=='AB',1,0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Glass':
        df = pd.read_csv('./datasets/glass.csv')
        unique_labels = sorted(df['target'].unique())
        label_map = {original: new for new, original in enumerate(unique_labels)}
        df['target'] = df['target'].map(label_map)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = df['target'].values
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Ionosphere':
        df = pd.read_csv('./datasets/Ionosphere.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        lower_bound = scaled_df.min()
        upper_bound = scaled_df.max()
        print(lower_bound, upper_bound)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Modeling':
        df = pd.read_csv('./datasets/User_Knowledge_Modeling.csv')
        unique_labels = sorted(df['target'].unique()) 
        label_map = {original: new for new, original in enumerate(unique_labels)}
        df['target'] = df['target'].map(label_map)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = df['target'].values
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Parkinson':
        df = pd.read_csv('./datasets/parkinsons.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        lower_bound = scaled_df.min()
        upper_bound = scaled_df.max()
        print(lower_bound, upper_bound)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Pima':
        df = pd.read_csv('./datasets/diabetes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Sonar':
        df = pd.read_csv('./datasets/sonar.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = df['target'].values
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    elif dataset_name == 'Wine':
        dataset = datasets.load_wine()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        lower_bound = scaled_df.min()
        upper_bound = scaled_df.max()
        print(lower_bound, upper_bound)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = dataset.target
        df_scaled['target'] = targets
        columns = df_scaled.columns
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, columns, np.unique(targets)
    else:
        print("Incorrect dataset name")
        
def parse_explanation(explanation, feature_names, epsilon=1e-6):
    bounds = [[0, 1] for _ in range(len(feature_names))]
    conditions = explanation

    for condition in conditions:
        condition_no_space = condition.replace(' ', '')  # for regex matching
        # Check for double inequality
        match = re.match(r'(\d+\.?\d*)\s*(<|<=)\s*([^\s<>]+)\s*(<|<=)\s*(\d+\.?\d*)', condition_no_space)
        
        if match:
            value_1, op1, feature_token, op2, value_2 = match.groups()
            value_1 = float(value_1)
            value_2 = float(value_2)
            lower_bound = value_1 if op1 == '<=' else value_1 + epsilon
            upper_bound = value_2 if op2 == '<=' else value_2
            upper_bound = upper_bound if op2 == '<=' else upper_bound - epsilon

            for idx, feature in enumerate(feature_names):
                if feature.replace(" ", "") in feature_token:
                    bounds[idx] = [lower_bound, upper_bound]
                    break
            continue  # go to next condition

        # Fallback to single operator logic
        for idx, feature in enumerate(feature_names):
            if feature in condition:
                cond_clean = condition.replace('<=', ' LESS_EQUAL ').replace('>=', ' GREATER_EQUAL ')
                cond_clean = cond_clean.replace('<', ' < ').replace('>', ' > ')
                tokens = cond_clean.split()

                tokens = ['<=' if token == 'LESS_EQUAL' else token for token in tokens]
                tokens = ['>=' if token == 'GREATER_EQUAL' else token for token in tokens]

                operator = None
                operator_pos = None
                for i, token in enumerate(tokens):
                    if token in ['>', '>=', '<', '<=']:
                        operator = token
                        operator_pos = i
                        break

                value = None
                if operator is not None and operator_pos is not None:
                    for i in range(operator_pos + 1, len(tokens)):
                        try:
                            value = float(tokens[i])
                            break
                        except ValueError:
                            continue

                if value is not None:
                    if operator == '>':
                        bounds[idx] = [value + epsilon, 1]
                    elif operator == '>=':
                        bounds[idx] = [value, 1]
                    elif operator == '<':
                        bounds[idx] = [0, value - epsilon]
                    elif operator == '<=':
                        bounds[idx] = [0, value]
                else:
                    print(f"Could not extract numeric value from condition: '{condition}'")

    return np.array(bounds)

In [3]:
from alibi.explainers import AnchorTabular

def run_anchors(dataset_name, epsilon = 1e-6, verbose=0):

    def predict_fn(x):
        return clf.predict(x, verbose=0)
    
    print(dataset_name)
    clf = tf.keras.models.load_model(f'new_models/{dataset_name}.h5', compile=False)
    print(f'Loaded model')
    test_dataset_df = pd.read_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv')
    X_train, X_test, y_train, y_test,feature_names, class_names = load_data(dataset_name)
    if 'target' in feature_names:
        feature_names = feature_names[feature_names != 'target']
    print(f'feature names:\n {feature_names}')
    print(f'class names:\n {class_names}')
    #predict_fn = lambda x: clf.predict(x)
    explainer = AnchorTabular(predict_fn, feature_names)
    explainer.fit(X_train, disc_perc=(25, 50, 75))
    coverages = []
    errors = []
    times = []
    rsums = []
    sizes = []
    for idx in range(len(X_test)):
        prediction = class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]]
        
        start = time.perf_counter()
        explanation = explainer.explain(X_test[idx], threshold=1)
        end =  time.perf_counter()
        times.append(end - start)
        size = 0
        for feature_name in feature_names:
            for anchor in explanation.anchor:
                if feature_name in re.split(r' <= | >= | < | > ', anchor):
                    size += 1
        bounds = parse_explanation(explanation.anchor, feature_names, epsilon)
        rsum = mymetrics.range_sum(bounds)
        
        coverage_df = mymetrics.calculate_coverage(test_dataset_df, bounds)
        my_coverage = len(coverage_df)
        error = len(coverage_df[coverage_df['target'] != prediction])
        original_instance_df = pd.DataFrame(X_test[idx].reshape(1, -1), columns=feature_names)
        original_instance_df['target'] = prediction
        instance_coverage = len(mymetrics.calculate_coverage(original_instance_df, bounds))
        if instance_coverage == 0:
            my_coverage += 1
        if verbose:
            print('\n\nPrediction: ', prediction)
            print('Anchor: %s' % (' AND '.join(explanation.anchor)))
            print(f'Bounds:\n {bounds}')
            print('Precision: %.2f' % explanation.precision)
            print('Coverage: %.2f' % explanation.coverage)
            print(f'Time: {times[-1]}')
            print(f'My_coverage: {my_coverage}')      
            if error > 0:    
                print(f'errors: {error}')
                print(f'explanation:\n {bounds}')
                display(coverage_df[coverage_df['target'] != prediction])
        coverages.append(my_coverage)
        errors.append(error)
        rsums.append(rsum)
        sizes.append(size)
    print('END')
    result_df = pd.DataFrame(columns=['coverage','errors','time'])
    result_df['coverage'] = coverages
    result_df['errors'] = errors
    result_df['time'] = times
    result_df['rsum'] = rsums
    result_df['size'] = sizes
    result_df.to_csv(f'./Anchors_results/{dataset_name}_results.csv',index=False)
    print(f'saved {dataset_name}_results.csv')
    #return coverages,errors, times
for dataset_name in ['Iris', 'Wine', 'Column', 'Pima', 'Parkinson', 'Breast_Cancer', 'Blood_Transfusion', 'Ionosphere', 'Glass', 'Climate', 'Modeling', 'Banknote', 'Sonar']:
    run_anchors(dataset_name = dataset_name)


Iris
Loaded model
feature names:
 Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')
class names:
 [0 1 2]


Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning th

END
saved Iris_results.csv
Wine
Loaded model
0.0 1.0
feature names:
 Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')
class names:
 [0 1 2]
END
saved Wine_results.csv
Column
Loaded model
Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True
feature names:
 Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis'],
      dtype='object')
class names:
 [0 1]


Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning th

END
saved Column_results.csv
Pima
Loaded model
Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True
feature names:
 Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')
class names:
 [0 1]


...

END
saved Glass_results.csv
Climate
Loaded model
feature names:
 Index(['vconst_corr', 'vconst_2', 'vconst_3', 'vconst_4', 'vconst_5',
       'vconst_7', 'ah_corr', 'ah_bolus', 'slm_corr', 'efficiency_factor',
       'tidal_mix_max', 'vertical_decay_scale', 'convect_corr', 'bckgrnd_vdc1',
       'bckgrnd_vdc_ban', 'bckgrnd_vdc_eq', 'bckgrnd_vdc_psim', 'Prandtl'],
      dtype='object')
class names:
 [0 1]
END
saved Climate_results.csv
Modeling
Loaded model
feature names:
 Index(['STG', 'SCG', 'STR', 'LPR', 'PEG'], dtype='object')
class names:
 [0 1 2 3]


Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning th

END
saved Modeling_results.csv
Banknote
Loaded model
feature names:
 Index(['variance', 'skewness', 'curtosis', 'entropy'], dtype='object')
class names:
 [0 1]


Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 1 precision constraint. Now returning th

END
saved Banknote_results.csv
Sonar
Loaded model
feature names:
 Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11',
       'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21',
       'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31',
       'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41',
       'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51',
       'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60'],
      dtype='object')
class names:
 [0 1]
END
saved Sonar_results.csv


In [4]:
# from alibi.explainers import AnchorTabular

# def predict_fn(x):
#     return clf.predict(x, verbose=0)

# idx = 0
# dataset_name = 'Iris'
# print(dataset_name)

# clf = tf.keras.models.load_model(f'new_models/{dataset_name}.h5', compile=False)
# test_dataset_df = pd.read_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv')
# X_train, X_test, y_train, y_test,feature_names, class_names = load_data(dataset_name)

# if 'target' in feature_names:
#     feature_names = feature_names[feature_names != 'target']
# print(f'feature names:\n {feature_names}')
# print(f'class names:\n {class_names}')

# #predict_fn = lambda x: clf.predict(x)
# explainer = AnchorTabular(predict_fn, feature_names)
# explainer.fit(X_train, disc_perc=(25, 50, 75))
# prediction = class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]]
# explanation = explainer.explain(X_test[idx], threshold=1)
# print('\n\nPrediction: ', prediction)
# print('Anchor: %s' % (' AND '.join(explanation.anchor)))